In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context('talk', font_scale=1.2, rc={'lines.linewidth': 3})
sns.set_style('whitegrid',
              {'grid.linestyle': ':', 'grid.color': 'red', 'axes.edgecolor': '0.5',
               'axes.linewidth': 1.2, 'legend.frameon': True})

import os
import pickle

import time

In [2]:
from scipy.constants import e, m_p, c

from scipy.constants import physical_constants

In [3]:
from cpymad.madx import Madx

import sixtracklib as pyst
import pysixtrack

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [5]:
import sys
sys.path.append('/home/oeftiger/gsi/git/python3/PyHEADTAIL/')
from PyHEADTAIL.particles import generators

PyHEADTAIL v1.13.5





In [6]:
nmass = physical_constants['atomic mass constant energy equivalent in MeV'][0] * 1e-3
nmass = 0.931494061 # MAD-X value

In [7]:
tune_range_qx = np.arange(18.55, 18.95 + 0.01, 0.01)
tune_range_qy = tune_range_qx

In [8]:
!mkdir -p results
!mkdir -p error_tables

# PySTL: read in lattice with errors in full definition

In [9]:
nturns = 20000 #2**16
npart = 1000

A = 238
Q = 28

Ekin_per_nucleon = 0.2e9 # in eV

###

mass = A * nmass * 1e9 * e / c**2 # in kg
charge = Q * e # in Coul

Ekin = Ekin_per_nucleon * A
p0c = np.sqrt(Ekin**2 + 2*Ekin*mass/e * c**2) # in eV

Etot = np.sqrt(p0c**2 + (mass/e)**2 * c**4) * 1e-9 # in GeV
p0 = p0c / c * e # in SI units
gamma = np.sqrt(1 + (p0 / (mass * c))**2)
beta = np.sqrt(1 - gamma**-2)

In [10]:
epsx_rms_fin = 35e-6 / 4 # geometrical emittances
epsy_rms_fin = 15e-6 / 4

limit_n_rms_x = 2
limit_n_rms_y = 2
limit_n_rms_z = 3.4

sig_z = 58 / 4. # in m
sig_dp = 0.5e-3

###

epsx_gauss = epsx_rms_fin * 1.778
epsy_gauss = epsy_rms_fin * 1.82

epsn_x = epsx_gauss * beta * gamma
epsn_y = epsy_gauss * beta * gamma

beta_z = sig_z / sig_dp

In [15]:
pyst.CudaTrackJob(elements, particles)

RuntimeError: Error while resetting the trackjob

In [ ]:
trackjob.

In [11]:
e_seed = 1

for qx in tune_range_qx:
    for qy in tune_range_qy:

        qqx, qqy = int(np.round((qx%1) * 100)), int(np.round((qy%1) * 100))

        filename_error_table = "./error_tables/errors_{qqx}_{qqy}_{eseed:d}".format(
                    qqx=qqx, qqy=qqy, eseed=e_seed)

        if os.path.exists('results/' + os.path.basename(filename_error_table) + '_done'):
            continue

        print ('\n\n\n=== Running at Qx = {:.2f} and Qy = {:.2f} ===\n\n\n'.format(qx, qy))

        madx = Madx()
        madx.options.echo = False
        madx.options.warn = False
        madx.options.info = False

        madx.call('./SIS100_RF_220618_9slices.thin.seq')

        madx.command.beam(particle='ion', mass=A*nmass, charge=Q, energy=Etot)

        madx.call('OpticsYEH_BeamParameters.str')
        madx.call('Coll+Errors+BeamDistr.madx')

        madx.input('''
        select, flag=seqedit, class=collimator;
        select, flag=seqedit, class=kicker;
        select, flag=seqedit, class=tkicker;
        select, flag=seqedit, class=elseparator;

        seqedit, sequence=SIS100RING;
            remove, element=selected;
            flatten;
        endedit;

        select, flag=seqedit, class=marker;
        seqedit, sequence=SIS100RING;
            remove, element=selected;
            install, element=SIS100RING$START, s=0;
            flatten;
        endedit;
        ''')

        madx.use(sequence='sis100ring')

        ### --> first match, then add errors, then TWISS!

        madx.input('''
            match, sequence=SIS100RING;
            global, sequence=SIS100RING, q1={qx}, q2={qy};
            vary, name=kqf, step=0.00001;
            vary, name=kqd, step=0.00001;
            lmdif, calls=500, tolerance=1.0e-10;
            endmatch;
        '''.format(qx=qx, qy=qy)
        )

        madx.command.eoption(add=True, seed=1)
        madx.command.exec('EA_EFCOMP_MH()')
        for s in range(1, 10):
            assert madx.command.exec(f'EA_rEFCOMP_QD({s},1)')

        twiss = madx.twiss();

        madx.command.select(flag='error', pattern='QD11..', class_='MULTIPOLE')
        madx.command.select(flag='error', pattern='QD12..', class_='MULTIPOLE')
        madx.command.select(flag='error', pattern='mh1', class_='MULTIPOLE')
        madx.command.select(flag='error', pattern='mh2', class_='MULTIPOLE')
        madx.command.esave(file=filename_error_table)

        madx.input('cavity_voltage = 58.2/1000/number_cavities;')

        madx.command.readtable(file=filename_error_table, table="errors")
        errors = madx.table.errors

        sis100 = madx.sequence.sis100ring

        # particle initialisation from pyheadtail

        D_x_0 = twiss['dx'][0] * beta
        D_y_0 = twiss['dy'][0] * beta

        np.random.seed(0)

        pyht_beam = generators.generate_Gaussian6DTwiss(
            npart, 1, charge, mass, twiss['s'][-1], gamma,
            twiss['alfx'][0], twiss['alfy'][0], twiss['betx'][0], twiss['bety'][0],
            1, epsn_x, epsn_y, 1,
            dispersion_x=D_x_0 if D_x_0 else None,
            dispersion_y=D_y_0 if D_y_0 else None,
            limit_n_rms_x=limit_n_rms_x**2, limit_n_rms_y=limit_n_rms_y**2, 
            limit_n_rms_z=limit_n_rms_z**2,
        )

        distribution_z_uncut = generators.gaussian2D(
            sig_z**2)
        is_accepted = generators.make_is_accepted_within_n_sigma(
            epsn_rms=sig_z,
            limit_n_rms=limit_n_rms_z,
        )
        distribution_z_cut = generators.cut_distribution(distribution_z_uncut, is_accepted)

        z, dp = distribution_z_cut(npart)
        pyht_beam.z, pyht_beam.dp = z, dp / beta_z

        ### PySixTrack, lattice transfer and preparation!

        pysixtrack_elements, _ = pysixtrack.Line.from_madx_sequence(
            sis100, exact_drift=True, install_apertures=True
        )

        pysixtrack_elements.remove_zero_length_drifts(inplace=True);
        pysixtrack_elements.merge_consecutive_drifts(inplace=True);

         # add alignment and multipole errors

        pysixtrack_elements.apply_madx_errors(error_table=errors)

        ### Load lattice into SixTrackLib

        elements = pyst.Elements.from_line(pysixtrack_elements)
        elements.BeamMonitor(num_stores=nturns);

        ### Transfer particles into SixTrackLib

        particles = pyst.Particles.from_ref(npart, p0c=p0c, mass0=A*nmass*1e9, q0=Q)

        particles.x[:] = pyht_beam.x
        particles.px[:] = pyht_beam.xp
        particles.y[:] = pyht_beam.y
        particles.py[:] = pyht_beam.yp
        particles.zeta[:] = pyht_beam.z
        particles.delta[:] = pyht_beam.dp

        particles.rpp[:] = 1. / (pyht_beam.dp + 1)

        restmass = mass * c**2
        restmass_sq = restmass**2
        E0 = np.sqrt((p0 * c)**2 + restmass_sq)
        p = p0 * (1 + pyht_beam.dp)
        E = np.sqrt((p * c)**2 + restmass_sq)
        particles.psigma[:] = (E - E0) / (beta * p0 * c)

        gammai = E / restmass
        betai = np.sqrt(1 - 1. / (gammai * gammai))
        particles.rvv[:] = betai / beta

        ### prepare trackjob in SixTrackLib

        trackjob = pyst.CudaTrackJob(elements, particles)
        
        

        print ('\n\n\n' + '+'*26 + '\n*** ready for tracking ***\n' + '+'*26 + '\n')

        ### TRACKING

        trackjob.track_until(nturns)
        trackjob.collect()

        store = {}
        filename_error_table = os.path.basename(filename_error_table)

        # statistics
        x = trackjob.output.particles[0].x.reshape((nturns, npart)).T
        store['std_x'] = np.mean(np.std(x, axis=0)[-50:])
        y = trackjob.output.particles[0].y.reshape((nturns, npart)).T
        store['std_y'] = np.mean(np.std(y, axis=0)[-50:])

        # losses
        pbuffer = trackjob.particles_buffer.get_object(0)
        np.save('results/' + filename_error_table + '_alive.npy', pbuffer.state)
        np.save('results/' + filename_error_table + '_lost_at_element.npy', 
                pbuffer.at_element[~pbuffer.state.astype(bool)])
        np.save('results/' + filename_error_table + '_lost_at_turn.npy',
                pbuffer.at_turn[~pbuffer.state.astype(bool)])

        store['losses'] = np.sum(pbuffer.state)

        # finish job
        pickle.dump(store, open('results/' + filename_error_table + '_summary.p', 'wb'))

        !touch "results/$filename_error_table"_done

        del trackjob, elements, particles




=== Running at Qx = 18.55 and Qy = 18.59 ===




  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2019.10.29 11:04:53      +
  ++++++++++++++++++++++++++++++++++++++++++++
START MATCHING

number of sequences: 1
sequence name: sis100ring
number of variables:    2
user given constraints: 2
total constraints:      2

START LMDIF:

Initial Penalty Function =   0.15132780E+02


call:       4   Penalty function =   0.20231065E-03
call:       7   Penalty function =   0.22333633E-10
 ++++++++++ LMDIF ended: converged successfully
call:       7   Penalty function =   0.22333633E-10

MATCH SUMMARY

Node_Name                  Constraint   Type  Target Value       Final Value        Penalty
--------------------------------------------------------------------------------------------------
Global constraint:         q1           4     1.855




=== Running at Qx = 18.58 and Qy = 18.55 ===




  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2019.10.29 11:05:39      +
  ++++++++++++++++++++++++++++++++++++++++++++
START MATCHING

number of sequences: 1
sequence name: sis100ring
number of variables:    2
user given constraints: 2
total constraints:      2

START LMDIF:

Initial Penalty Function =   0.15088675E+02


call:       4   Penalty function =   0.14499471E-03
call:       7   Penalty function =   0.31207536E-09
call:      10   Penalty function =   0.18431884E-14
 ++++++++++ LMDIF ended: converged successfully
call:      10   Penalty function =   0.18431884E-14

MATCH SUMMARY

Node_Name                  Constraint   Type  Target Value       Final Value        Penalty
--------------------------------------------------------------------------------------------------

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/HPC/oeftiger/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-94907d98715c>", line 120, in <module>
    pysixtrack_elements.apply_madx_errors(error_table=errors)
  File "/home/HPC/oeftiger/aoeftiger/pysixtrack/pysixtrack/line.py", line 421, in apply_madx_errors
    self.add_tilt_error_to(element, angle=dpsi * 180 / np.pi)
  File "/home/HPC/oeftiger/aoeftiger/pysixtrack/pysixtrack/line.py", line 334, in add_tilt_error_to
    idx_el = self.elements.index(element)
  File "<string>", line 1, in __eq__
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/HPC/oeftiger/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrup

KeyboardInterrupt: 

In [23]:
np.genfromtxt('./Vera_Tunes_Emittances.dat', max_rows=1, dtype=str)

array(['QXsoll', 'QYsoll', 'ERROR_SET', 'QXistNoSC', 'QYistNoSC',
       'EMITXi', 'EMITYi', 'EMITXe', 'EMITYe', 'PARTICLES_LEFT'],
      dtype='<U14')

In [24]:
np.genfromtxt('./Vera_Tunes_Emittances.dat', skip_header=1, max_rows=1)

array([5.50000000e+01, 5.50000000e+01, 1.00000000e+00, 1.85500000e+01,
       1.85500000e+01, 8.75204497e-06, 3.69995050e-06, 5.10266004e-06,
       5.10787475e-06, 7.95000000e+02])